# Relax Inc. Take Home Challenge
## *The goal is to identify which factors predict future user adoption*

In [1]:
# imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

/home/aregel/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [17]:
# load data into dataframes
user_engagement = pd.read_csv('takehome_user_engagement.csv')
users = pd.read_csv('takehome_users.csv', encoding = 'iso-8859-1')

In [25]:
# change dates to datetime objects
users['creation_time'] = pd.to_datetime(users.creation_time)
users['last_session_creation_time'] = pd.to_datetime(users.last_session_creation_time)
user_engagement['time_stamp'] = pd.to_datetime(user_engagement.time_stamp)

In [21]:
pd.pivot_table(user_engagement.sort_values('time_stamp'), index = ['user_id', 'time_stamp'])

visited
user_id time_stamp                  
1       2014-04-22 03:53:30        1
2       2013-11-15 03:45:04        1
        2013-11-29 03:45:04        1
        2013-12-09 03:45:04        1
        2013-12-25 03:45:04        1
        2013-12-31 03:45:04        1
        2014-01-08 03:45:04        1
        2014-02-03 03:45:04        1
        2014-02-08 03:45:04        1
        2014-02-09 03:45:04        1
        2014-02-13 03:45:04        1
        2014-02-16 03:45:04        1
        2014-03-09 03:45:04        1
        2014-03-13 03:45:04        1
        2014-03-31 03:45:04        1
3       2013-03-19 23:14:52        1
4       2013-05-22 08:09:28        1
5       2013-01-22 10:14:20        1
6       2013-12-19 03:37:06        1
7       2012-12-20 13:24:32        1
10      2013-01-16 22:08:03        1
        2013-01-22 22:08:03        1
        2013-01-30 22:08:03        1
        2013-02-04 22:08:03        1
        2013-02-06 22:08:03        1
        2013-02-14 22:08:03        1
        2013-02-17 22:08:03        1
        2013-02-19 22:08:03        1
        2013-02-26 22:08:03        1
        2013-03-01 22:08:03        1
...                              ...
11988   2014-04-28 11:04:47        1
        2014-04-29 11:04:47        1
        2014-04-30 11:04:47        1
        2014-05-04 11:04:47        1
        2014-05-08 11:04:47        1
        2014-05-14 11:04:47        1
        2014-05-17 11:04:47        1
        2014-05-18 11:04:47        1
        2014-05-19 11:04:47        1
        2014-05-20 11:04:47        1
        2014-05-21 11:04:47        1
        2014-05-23 11:04:47        1
        2014-05-24 11:04:47        1
        2014-05-26 11:04:47        1
        2014-05-27 11:04:47        1
        2014-06-01 11:04:47        1
11989   2013-09-07 11:24:33        1
11990   2013-07-06 21:00:48        1
11991   2014-04-20 14:22:45        1
        2014-04-25 14:22:45        1
        2014-04-28 14:22:45        1
        2014-05-17 14:22:45        1
        2014-05-30 14:22:45        1
11992   2014-04-11 19:39:56        1
11995   2013-10-22 13:33:17        1
11996   2013-09-06 06:14:15        1
11997   2013-01-15 18:28:37        1
11998   2014-04-27 12:45:16        1
11999   2012-06-02 11:55:59        1
12000   2014-01-26 08:57:12        1

[207917 rows x 1 columns]